In [1]:
# Set up environment variables
import os
# Test the model
from openai import OpenAI
from dotenv import load_dotenv
import http


from crewai_tools import (  # pyright: ignore[reportMissingImports]
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

from typing import Type, List
from pydantic import BaseModel, Field 
from crewai.tools import BaseTool   # pyright: ignore[reportMissingImports]
from crewai import LLM, Agent, Task, Crew  # pyright: ignore[reportMissingImports]

import arxiv  # pyright: ignore[reportMissingImports]
import time
import datetime

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

In [2]:
# Test if the keys are working

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

response = client.responses.create(
    input="What are transformers in Deep Learning. Explain in 2 lines only",
    model="llama-3.1-8b-instant",
)
print(response.output_text)


Transformers in Deep Learning are a type of neural network architecture that uses self-attention mechanisms to process sequential data, such as natural language or audio signals, enabling parallelization and contextual understanding. They are particularly useful for tasks like language translation, text summarization, and speech recognition.


In [3]:
conn = http.client.HTTPSConnection("google.serper.dev")
payload = ''
headers = {}
conn.request("GET", "/search?q=cat&apiKey=616509eec3f7c2d80906ae20249417939e52970a", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"searchParameters":{"q":"cat","type":"search","engine":"google"},"organic":[{"title":"Cat","link":"https://en.wikipedia.org/wiki/Cat","snippet":"The cat (Felis catus), also called domestic cat and house cat, is a small carnivorous mammal. It is an obligate carnivore, requiring a predominantly meat-based ...","position":1},{"title":"Caterpillar: Products & Services – North America | Cat","link":"https://www.cat.com/en_US.html","snippet":"Cat® equipment and services are backed by the largest, most experienced dealer network in the world. Get the job done with our products, technology and ...","position":2},{"title":"ALL THE CATS, EXPLAINED","link":"https://www.youtube.com/watch?v=UWagM1FBB4c","snippet":"Wild felines come in all shapes and sizes and live all over the world plus there are more than half a billion domestic cats.","date":"2 months ago","position":3},{"title":"Domestic cat","link":"https://www.nationalgeographic.com/animals/mammals/facts/domestic-cat","snippet":"As mostly no

In [4]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./your-directory') # Tool for reading documents from a specified directory
file_tool = FileReadTool() # Tool for reading individual files
search_tool = SerperDevTool() # Tool for performing web searches using the Serper API
web_rag_tool = WebsiteSearchTool() # Tool for searching and extracting information from websites

In [5]:
class FetchArxivPapersInput(BaseModel):
    """Input schema for FetchArxivPapersTool."""
    target_date: datetime.date = Field(..., description="Target date to fetch papers for.")

class FetchArxivPapersTool(BaseTool):
    name: str = "fetch_arxiv_papers"
    description: str = "Fetches all ArXiv papers from selected categories submitted on the target date."
    args_schema: Type[BaseModel] = FetchArxivPapersInput

    def _run(self, target_date: datetime.date) -> List[dict]:
        # List of AI-related categories. 
        # You can also include ["cs.AI", "cs.LG", "cs.CV", "cs.MA", "cs.RO"]
        AI_CATEGORIES = ["cs.CL"]

        # Define the date range for the target date
        start_date = target_date.strftime('%Y%m%d%H%M')
        end_date = (target_date + datetime.timedelta(days=1)).strftime('%Y%m%d%H%M')

        # Initialize the ArXiv client
        client = arxiv.Client(
            page_size=100,  # Fetch 100 results per page
            delay_seconds=3  # Delay between requests to respect rate limits
        )

        all_papers = []

        for category in AI_CATEGORIES:
            print(f"Fetching papers for category: {category}")

            search_query = f"cat:{category} AND submittedDate:[{start_date} TO {end_date}]"

            search = arxiv.Search(
                query=search_query,
                sort_by=arxiv.SortCriterion.SubmittedDate,
                max_results=None  # Fetch all results
            )

            # Collect results for the category
            category_papers = []
            for result in client.results(search):
                category_papers.append({
                    'title': result.title,
                    'authors': [author.name for author in result.authors],
                    'summary': result.summary,
                    'published': result.published,
                    'url': result.entry_id
                })

                # Delay between requests to respect rate limits
                time.sleep(3)

            print(f"Fetched {len(category_papers)} papers from {category}")
            all_papers.extend(category_papers)

        return all_papers

In [6]:
arxiv_search_tool = FetchArxivPapersTool()

In [7]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.5,
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

In [8]:
researcher = Agent(
    role = "Senior Researcher",
    goal = "Find the top 10 papers from the search results from ArXiv on {date}."
            "Rank them appropirately.",
    backstory = "You are a senior researcher with a deep understanding of all topics in AI and AI research."
                "You are able to identify the best research papers based on the title and abstract.",
    verbose = True,
    llm = llm,
    tools = [arxiv_search_tool],
)

In [9]:
# Agent 2: Frontend Engineer

frontend_engineer = Agent(
    role = "Senior Frontend & AI Engineer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent frontend engineer writing HTML and CSS with decades of experience."
                "You have also been working with AI for decades and understand it well.",
    verbose = True,
    llm = llm
)

In [10]:
# Task for ArXiv Researcher

research_task = Task(
    description = (" Find the top 10 research papers from the search results from ArXiv on {date}."),
    expected_output = (
        "A list of top 10 research papers with the following information in the following format:"
        "- Title"
        "- Authors"
        "- Abstract"
        "- Link to the paper"
    ),
    agent = researcher,
    human_input = True,
)

In [11]:
# Task for Frontend Engineer

reporting_task = Task(
    description = ("Compile the results into a detailed report in a HTML file."),
    expected_output = (
        "An HTML file with the results in the following format:"
        "Top 10 AI Research Papers published on {date}"
        "- Title (which on clicking opens the paper in a new tab)"
        "- Authors"
        "- Short summary of the abstract (2-4 sentences)"
    ),
    agent = frontend_engineer,
    context = [research_task],
    output_file = "ai_research_report.html",
    human_input = True,
)

In [12]:
arxiv_research_crew = Crew(
    agents = [researcher, frontend_engineer],
    tasks = [research_task, reporting_task],
    verbose = True,
)

In [ ]:
crew_inputs = {
    "date" : "2025-03-12"
}

result = arxiv_research_crew.kickoff(inputs = crew_inputs)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e9c2ed42-180a-4ce8-a90d-7ca704671d3b                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:  Find the top 10 research papers from the search results from ArXiv on 2025-03-12.                       │
│  ID: b4d33a99-1ea6-4ec6-8c55-f1bca380fcf1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Researcher                                                                                       │
│                                                                                                                 │
│  Task:  Find the top 10 research papers from the search results from ArXiv on 2025-03-12.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[CrewAIEventsBus] Warning: Event pairing mismatch. 'tool_usage_finished' closed 'llm_call_started' (expected 
'tool_usage_started')

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: fetch_arxiv_papers                                                                                       │
│  Args: {'target_date': '2025-03-12'}                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 



┌───────────────────────── Tracing Preference Saved ──────────────────────────┐
│                                                                             │
│  Info: Tracing has been disabled.                                           │
│                                                                             │
│  Your preference has been saved. Future Crew/Flow executions will not       │
│  collect traces.                                                            │
│                                                                             │
│  To enable tracing later, do any one of these:                              │
│  • Set tracing=True in your Crew/Flow code                                  │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file              │
│  • Run: crewai traces enable                                                │
│                                                                             │
└─────────────────────────────────────

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e9c2ed42-180a-4ce8-a90d-7ca704671d3b                                                                           │
│  Final Output: ```html                                                                                          │
│  <!DOCTYPE html>                                                                                                │
│  <html>                                                                                                         │
│  <head>                                                                                                         │
│          <title>Top 10 AI Research Papers</title>                                                               │
│          <style>                                                                                                │
│                  body {                                                                                         │
│                          font-family: Arial, sans-serif;                                                        │
│                  }                                                                                              │
│                  .paper {                                                                                       │
│                          margin-bottom: 20px;                                                                   │
│                          padding: 10px;                                                                         │
│                          border: 1px solid #ddd;                                                                │
│                          border-radius: 10px;                                                                   │
│                          box-shadow: 0 0 10px rgba(0,0,0,0.1);                                                  │
│                  }                                                                                              │
│                  .paper h2 {                                                                                    │
│                          margin-top: 0;                                                                         │
│                  }                                                                                              │
│                  .paper a {                                                                                     │
│                          text-decoration: none;                                                                 │
│                          color: #337ab7;                                                                        │
│                  }                                                                                              │
│                  .paper a:hover {                                                                               │
│                          color: #23527c;                                                                        │
│                  }                                                                                              │
│          </style>                                                                                               │
│  </head>                                              

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯